In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("FileFormat")

from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc= spark.sparkContext

22/03/27 21:47:21 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/27 21:47:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ce44b806-202d-4278-93f8-e011bc0946fa;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 595ms :: artifacts dl 10ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [3]:
movieDf = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load("hdfs://localhost:9000/movies")

In [4]:
ratingDf = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load("hdfs://localhost:9000/ratings")

In [5]:
movieDf.write.mode("overwrite")\
            .json("hdfs://localhost:9000/movies-json")
                    

In [6]:
ratingDf.write.mode("overwrite")\
            .json("hdfs://localhost:9000/ratings-json")

In [7]:
movieJsonDf = spark.read.format("json")\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/movies-json")

movieJsonDf.printSchema()
movieJsonDf.show(5)

root
 |-- genres: string (nullable = true)
 |-- movieId: long (nullable = true)
 |-- title: string (nullable = true)

+--------------------+-------+--------------------+
|              genres|movieId|               title|
+--------------------+-------+--------------------+
|Adventure|Animati...|      1|    Toy Story (1995)|
|Adventure|Childre...|      2|      Jumanji (1995)|
|      Comedy|Romance|      3|Grumpier Old Men ...|
|Comedy|Drama|Romance|      4|Waiting to Exhale...|
|              Comedy|      5|Father of the Bri...|
+--------------------+-------+--------------------+
only showing top 5 rows



In [8]:
ratingJsonDf = spark.read.format("json")\
                .option("inferSchema", True)\
                .load("hdfs://localhost:9000/ratings-json")

ratingJsonDf.printSchema()
ratingJsonDf.show(5)

root
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- userId: long (nullable = true)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|      1|   4.0|964982703|     1|
|      3|   4.0|964981247|     1|
|      6|   4.0|964982224|     1|
|     47|   5.0|964983815|     1|
|     50|   5.0|964982931|     1|
+-------+------+---------+------+
only showing top 5 rows



In [9]:
movieDf.write.mode("overwrite")\
            .parquet("hdfs://localhost:9000/movies-parquet")

ratingDf.write.mode("overwrite")\
            .parquet("hdfs://localhost:9000/ratings-parquet")

In [10]:
movieParquetDf = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/movies-parquet")

movieParquetDf.printSchema()
movieParquetDf.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [11]:
ratingParquetDf = spark.read.format("parquet")\
                .load("hdfs://localhost:9000/ratings-parquet")

ratingParquetDf.printSchema()
ratingParquetDf.show(5)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [12]:
movieDf.write.mode("overwrite")\
            .orc("hdfs://localhost:9000/movies-orc")

ratingDf.write.mode("overwrite")\
            .orc("hdfs://localhost:9000/ratings-orc")

In [13]:
movieOrcDf = spark.read.format("orc")\
                .load("hdfs://localhost:9000/movies-orc")

movieOrcDf.printSchema()
movieOrcDf.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [14]:
ratingOrcDf = spark.read.format("orc")\
                .load("hdfs://localhost:9000/ratings-orc")

ratingOrcDf.printSchema()
ratingOrcDf.show(5)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

